# Calendar API

Useful for working out consulting hours and task descriptions.

## Requires

Google calender where you input your events in an accessible calendar in with event summaries in the form:

    ClientName: some summary text

The output in the last cell of this notebook will then be similar to:

    Getting consulting events in date range for ClientName:

    2019-07-26 1.0 ClientName: some summary text
    2019-08-02 5.0 ClientName: a longer task
    Some notes I wrote in the description of this calendar event 
    * maybe a point
    * or two

    Summary for ClientName:

        Date from =  2019-07-24
          Date To =  2019-08-27
      Total hours =  1.0

Where each event is listed with the date, number of hours and the summary text. If descriptions exist they are listed under the relevant summary

## Instructions for connecting your calendar

The following link tells you more information about the basis code I used for this notebook and how to install the packages and connect your own calendar.

https://developers.google.com/calendar/quickstart/python

In [ ]:
from __future__ import print_function
import datetime
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request


In [ ]:
# If modifying these scopes, delete the file token.pickle.
SCOPES = ['https://www.googleapis.com/auth/calendar.readonly']

# calendar id(s) - I usually have my consulting work in a non-primary calendar
calendar_id = 'primary'

In [ ]:
def initCalendarService():
    """Shows basic usage of the Google Calendar API.
    Prints the start and name of the next 10 events on the user's calendar.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)

    service = build('calendar', 'v3', credentials=creds)
    return service

datetime.fromisoformat(date_string)

    Return a datetime corresponding to a date_string in one of the formats emitted by date.isoformat() and datetime.isoformat(). Specifically, this function supports strings in the format(s) YYYY-MM-DD[*HH[:MM[:SS[.fff[fff]]]][+HH:MM[:SS[.ffffff]]]], where * can match any single character.

In [ ]:
def getDurationHours(startDate='2019-07-15T13:10:00+10:00', endDate='2019-07-15T15:10:00+10:00'):
    from datetime import datetime
    duration = datetime.fromisoformat(endDate) - datetime.fromisoformat(startDate)
    return round(duration.seconds/60/60, 1)

def convertDateToText(startDate='2019-07-15T13:10:00+10:00'):
    return startDate.split('T')[0]



## Get the next 10 events

For more info on events

https://developers.google.com/calendar/v3/reference/events




In [ ]:
service = initCalendarService()

# Call the Calendar API
now = datetime.datetime.utcnow().isoformat() + 'Z' # 'Z' indicates UTC time

print('Getting the upcoming 10 events from consulting')
events_result = service.events().list(calendarId=calendar_id, timeMin=now,
                                    maxResults=10, singleEvents=True,
                                    orderBy='startTime').execute()
events = events_result.get('items', [])

if not events:
    print('No upcoming events found.')
for event in events:
    startDate = event['start'].get('dateTime', event['start'].get('date'))
    endDate = event['end'].get('dateTime', event['end'].get('date'))
    print(type(endDate))
    print(startDate, endDate, event['summary'])

List options from https://developers.google.com/calendar/v3/reference/events/list

    timeMax 	datetime 	Upper bound (exclusive) for an event's start time to filter by. Optional. The default is not to filter by start time. Must be an RFC3339 timestamp with mandatory time zone offset, for example, 2011-06-03T10:00:00-07:00, 2011-06-03T10:00:00Z. Milliseconds may be provided but are ignored. If timeMin is set, timeMax must be greater than timeMin.

    timeMin 	datetime 	Lower bound (exclusive) for an event's end time to filter by. Optional. The default is not to filter by end time. Must be an RFC3339 timestamp with mandatory time zone offset, for example, 2011-06-03T10:00:00-07:00, 2011-06-03T10:00:00Z. Milliseconds may be provided but are ignored. If timeMax is set, timeMin must be smaller than timeMax. 

In [ ]:
timeMinText = '2019-07-24T00:00:00+10:00' # AEST at end
timeMaxText = '2019-08-27T00:59:00+10:00'
keyWord = 'ClientName:'

print('Getting consulting events in date range for', keyWord)
print()
events_result = service.events().list(calendarId=calendar_id, 
                                      timeMin=timeMinText, timeMax=timeMaxText,
                                      singleEvents=True, orderBy='startTime').execute()
events = events_result.get('items', [])

total_hours = 0.0
if not events:
    print('No events found.')
for event in events:
    startDate = event['start'].get('dateTime', event['start'].get('date'))
    endDate = event['end'].get('dateTime', event['end'].get('date'))
    summaryText = event['summary']
    try:
        descriptionText = event['description']
    except:
        descriptionText = ''
# https://developers.google.com/calendar/v3/reference/events/list
    if(keyWord in summaryText):
        duration = getDurationHours(startDate, endDate)
        total_hours += duration
        startDateText = convertDateToText(startDate)
        print(startDateText, duration, summaryText, descriptionText)
        last_event = event.copy()
        
# summary
print('\nSummary for', keyWord)
print('    Date from = ', convertDateToText(timeMinText))
print('      Date To = ', convertDateToText(timeMaxText))
print('  Total hours = ', total_hours)